#### Import packages

In [53]:
import pandas as pd
import numpy as np
import cv2 # installed from opencv-python
import os
import shutil

#### Define functions

In [64]:
def fetch_data(type):
    """Returns a dataframe of either 'train' or 'val' data including depth maps in png format."""
    df = pd.read_csv(type + '_outdoor.csv', header=None)
    df = df.drop(df.columns[[2, 3]], axis=1)
    df.columns = ['image', 'depth']

    return df

def mkdata(type):
    """Creates the new dataset txt file of either 'train' or 'val'."""
    raw = []
    depth = []
    camera = []

    for root, dirs, files in os.walk(type):
        for fileName in files:
            dest = os.path.join(root[len(type):], fileName).replace('\\', '/')[1:]
            depth.append(dest)
            raw.append('2011_09_26/' + dest.split('/')[0] + '/' + dest.split('/')[3] + '/data/' + dest.split('/')[4])
            camera.append('886.81')

    with open(type + "_dataset.txt", 'w') as outfile:
        for elem in zip(raw, depth, camera):
            outfile.write(str(elem[0]) + " " + elem[1] + " " + elem[2] +"\n")
    return

def mkraw(type):
    """Creates the raw folder structure of either 'train' or 'val'."""
    df = fetch_data(type)
    dest = []
    files = []

    f = open(type + "_dataset.txt","r")
    lines = f.readlines()
    for x in lines:
        temp = x.split(' ')[0]
        dest.append("../DIODEASKITTI/" + "/".join(temp.split("/", 4)[:4]))
        files.append (temp.split("/")[4])
    f.close()

    for filePath in dest:
        if not os.path.exists(filePath):
            os.makedirs(filePath)

    for k in range(len(files)):
        shutil.copy(df['image'][k], dest[k])
        os.rename(dest[k]+'\\'+df['image'][k].split('/')[5], dest[k]+'\\'+files[k])
    return

def mkdepth(type):
    """Creates the depth map folder structure of either 'train' or 'val'."""
    df = fetch_data(type)
    dest = []
    files = []

    f = open(type + "_dataset.txt","r")
    lines = f.readlines()
    for x in lines:
        temp = x.split(' ')[1]
        dest.append("../DIODEASKITTI/data_depth_annotated/" + "/".join(temp.split("/", 4)[:4]))
        files.append (temp.split("/")[4])
    f.close()

    for filePath in dest:
        if not os.path.exists(filePath):
            os.makedirs(filePath)

    for k in range(len(files)):
        depth = np.load(df['depth'][k])
        is_success, im_buf_arr = cv2.imencode('.png', depth)
        im_buf_arr.tofile(dest[k]+'\\'+files[k])
    return

#### Create new dataset files

In [59]:
os.chdir('D:\\Informatics\\Μεταπτυχιακό\\Εξάμηνο Γ\\dataset\\KITTI')

# call the function for 'train' and 'val' data
mkdata('train')
mkdata('val')

#### Create raw folder structure

In [65]:
os.chdir('D:\\Informatics\\Μεταπτυχιακό\\Εξάμηνο Γ\\dataset\\DIODE')

# call the function for 'train' and 'val' data
mkraw('train')
mkraw('val')

#### Create depth map folder structure

In [67]:
os.chdir('D:\\Informatics\\Μεταπτυχιακό\\Εξάμηνο Γ\\dataset\\DIODE')

# call the function for 'train' and 'val' data
mkdepth('train')
mkdepth('val')